In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
from WineQualityModelService import WineQualityModelService

In [2]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

Download dataset

In [3]:
# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
data = pd.read_csv(csv_url, sep=";")

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

In [4]:
# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

alpha = 0.5
l1_ratio = 0.5

Fit linear model

In [5]:
lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
lr.fit(train_x, train_y)

predicted_qualities = lr.predict(test_x)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7101081210266617
  MAE: 0.5855972338594355
  R2: 0.16569565925212437


Package with BentoML

In [6]:
bento_lr = WineQualityModelService()
bento_lr.pack("model", lr)
bento_lr.save()

[2021-05-04 12:53:30,645] INFO - BentoService bundle 'WineQualityModelService:20210504125324_760D25' saved to: /home/dlowl/bentoml/repository/WineQualityModelService/20210504125324_760D25


'/home/dlowl/bentoml/repository/WineQualityModelService/20210504125324_760D25'

In [9]:
print(test_x.iloc[1].to_json())
print(test_y.iloc[1].to_json())

{"fixed acidity":7.9,"volatile acidity":0.29,"citric acid":0.49,"residual sugar":2.2,"chlorides":0.096,"free sulfur dioxide":21.0,"total sulfur dioxide":59.0,"density":0.99714,"pH":3.31,"sulphates":0.67,"alcohol":10.1}
{"quality":6}
